In [6]:
from summarytools import dfSummary

import pandas as pd

In [7]:
channels = pd.read_csv("../databases/channels.csv", sep=",")
deliveries = pd.read_csv("../databases/deliveries.csv", sep=",")
drivers = pd.read_csv("../databases/drivers.csv", sep=",")
hubs = pd.read_csv("../databases/hubs.csv", sep=",", encoding='latin1')
orders = pd.read_csv("../databases/orders.csv", sep=",")
payments = pd.read_csv("../databases/payments.csv", sep=",")
stores = pd.read_csv("../databases/stores.csv", sep=",", encoding='latin1')

In [8]:
dfs = {"channels":channels, 
       "deliveries":deliveries,
       "drivers":drivers,
       "hubs":hubs,
       "orders":orders,
       "payments":payments,
       "stores":stores}

for name, df in dfs.items():
    print(f"\n{name}:\n")
    display(df.head(2))


channels:



,channel_id,channel_name,channel_type
0,1,OTHER PLACE,OWN CHANNEL
1,2,PHONE PLACE,OWN CHANNEL



deliveries:



,delivery_id,delivery_order_id,driver_id,delivery_distance_meters,delivery_status
0,2174658,68413340,8378.0,5199.0,DELIVERED
1,2174660,68414309,2473.0,410.0,DELIVERED



drivers:



...

,order_id,store_id,channel_id,payment_order_id,delivery_order_id,order_status,order_amount,order_delivery_fee,order_delivery_cost,order_created_hour,...,order_moment_delivering,order_moment_delivered,order_moment_finished,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
0,68405119,3512,5,68405119,68405119,CANCELED,62.7,0.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68405123,3512,5,68405123,68405123,CANCELED,62.7,0.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



payments:



,payment_id,payment_order_id,payment_amount,payment_fee,payment_method,payment_status
0,4427917,68410055,118.44,0.0,VOUCHER,PAID
1,4427918,68410055,394.81,7.9,ONLINE,PAID



stores:



,store_id,hub_id,store_name,store_segment,store_plan_price,store_latitude,store_longitude
0,3,2,CUMIURI,FOOD,0.0,NaN,NaN
1,6,3,PIMGUCIS DA VIVA,FOOD,0.0,-30.037415,-51.20352


**Dados vazios?**

In [9]:
orders.shape, orders.isnull().sum()

((368999, 29),
 order_id                                  0
 store_id                                  0
 channel_id                                0
 payment_order_id                          0
 delivery_order_id                         0
 order_status                              0
 order_amount                              0
 order_delivery_fee                        0
 order_delivery_cost                    7205
 order_created_hour                        0
 order_created_minute                      0
 order_created_day                         0
 order_created_month                       0
 order_created_year                        0
 order_moment_created                      0
 order_moment_accepted                  9461
 order_moment_ready                    25106
 order_moment_collected                42894
 order_moment_in_expedition            67429
 order_moment_delivering               25316
 order_moment_delivered               349398
 order_moment_finished                 1

In [10]:
channels.shape, channels.isnull().sum()

((40, 3),
 channel_id      0
 channel_name    0
 channel_type    0
 dtype: int64)

In [11]:
stores.shape, stores.isnull().sum()

((951, 7),
 store_id              0
 hub_id                0
 store_name            0
 store_segment         0
 store_plan_price    115
 store_latitude       16
 store_longitude      16
 dtype: int64)

In [12]:
hubs.shape, hubs.isnull().sum()

((32, 6),
 hub_id           0
 hub_name         0
 hub_city         0
 hub_state        0
 hub_latitude     0
 hub_longitude    0
 dtype: int64)

In [13]:
deliveries.shape, deliveries.isnull().sum()

((378843, 5),
 delivery_id                     0
 delivery_order_id               0
 driver_id                   15886
 delivery_distance_meters       73
 delivery_status                 0
 dtype: int64)

In [14]:
drivers.shape, drivers.isnull().sum()

((4824, 3),
 driver_id       0
 driver_modal    0
 driver_type     0
 dtype: int64)

**Cruzamento de Bases**

In [15]:
deliveriesNaN = deliveries.dropna(subset="driver_id")
deliveriesNaN["driver_id"] = deliveriesNaN["driver_id"].astype("int64")

database = orders.merge(channels, how="left", on="channel_id") \
                 .merge(stores, how="left", on="store_id") \
                 .merge(hubs, how="left", on="hub_id") \
                 .merge(deliveriesNaN, how="left", on="delivery_order_id") \
                 .merge(drivers, how="left", on="driver_id")

C:\Users\Computador\AppData\Local\Temp\ipykernel_4392\3027242523.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deliveriesNaN["driver_id"] = deliveriesNaN["driver_id"].astype("int64")


In [16]:
dfSummary(database)

C:\Users\Computador\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\summarytools\summarytools.py:124: RuntimeWarning: divide by zero encountered in scalar divide
  stats += f"<br>IQR (CV) : {x.quantile(0.75) - x.quantile(0.25):.1f} ({x.mean()/x.std():.1f})"


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,order_id[int64],Mean (sd) : 82311715.8 (7010887.3)min < med < max:68405119.0 < 83228842.0 < 93139817.0IQR (CV) : 11556068.0 (11.7),"368,999 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACl0lEQVR4nO3cPY7aQBiA4W/Cspgf2eJH0HGBlVIg0XCAlDlsjrAFxdbcgA4JWUY4a2GMU0faBGzPwHzoffrBU7xeZsZmTVmWAvju26MnANzi5dETgF3GmK6IvNYcfirL8tPmfGwh1CdijOmOx+OfYRgO64w/HA6xMeaXj7ES6nN5DcNwuFqtPqMoyqoMTJIkWK/Xw/1+/yoihAr3oijKptPp7xpDu9YnYwmbKahAqFCBUKECoUIFQoUKhAoVOJ6CNS6fihEqrHD9VIxQYYvTp2KECqtcPRVjMwUVCBUqECpUIFSowGbKMw3PIsPL5dK2OR9fEKpHmp5F5nkeFEXxlmXZu4hU3nkXRdEWkdAYU+fyTm8SQvVL7bNIEZHtdjvcbDa98/ncqjo2TdN2nuff5/N5q9VqVb5205vkGkL1UN2zyDiOa7+hfzqdXoIg6C0Wi2w2m8VVxze5SW5BqPhLv9+/+01yC0J1oMGG6Gk3Q00RqmVNNkSu13maEap9tTdErtd5mhGqI3U2RK7XeZoR6j+wzvQLoX6BdaZ/CPVrrDM987Sh2nhmHkVRzDrTD16H2iC2YDQa/YiiqF/nunx9+8fbUJuuE0Xkbblcfkwmk0PV8Xx9+8fbUMXCOjEIgtzHx4GozmmorBNhi7NQH/1uJZ7L1VCbHHwPBoPparU63PvdSjyf/4Zq4+C70+m8s05EU9f+onLwDS/ctEblBQs8Gj+XhgqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSrc9L+nkiQJqn7w8XgMRETSNA12u13vnuO1XlvrvJuOv6WvP0dGfwzkCNULAAAAAElFTkSuQmCC"">",0(0.0%)
2,store_id[int64],Mean (sd) : 1202.2 (1152.2)min < med < max:3.0 < 707.0 < 4679.0IQR (CV) : 1141.0 (1.0),951 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC2klEQVR4nO3dMW7iQBSA4feWgOxEGQsRQecLrLRFSprcIIfdNt0WcAFuQIOQkAHBBkEI2ySrLaKAZ3Dwm/2/mvG4+IWxJZ71cDgIUHffLn0CwCmujn1AVVMRaXkef3s4HJ491wJ/fRqqqqadTufROdf2Ofh8Pl+p6pOIbLzOjtDx5tg3ass51+73+89ZlpWKbTKZ3A6Hw4c8z+8ajYZXqMvlslDVn8SKo5d+EZEsyzbdbvd3mQMXRZEmSXJ9f3+/6fV6RdkTWywWyWAwaM9ms5aIEOp/7qRQQ9zc3JSO/B/pWU8GZnHXDxMIFSYQKkwgVJhAqDCBUGECocIEQoUJhAoTCBUmECpMIFSYQKgwgVBhAqHCBEKFCYQKEwgVJhAqTCBUmECoMIFQYULlf5cOsd/vmyLiVNVnOVNWIlLbUNfrdXO32/3I87zhM2mFKStxqW2o2+32ynfSClNW4lPbUN8FTFphykpEuJmCCYQKEwgVJhAqTCBUmECoMIFQYQKhwgRChQmEChMIFSYQKkwgVJhAqDCBUGECocIEQoUJhAoTCBUmECpMIFSYQKgwofZ/l74UVU1FpOW53HtKS+C+QXvXGaF+QFXTTqfz6Jxr+6z3ndISum/I3nVHqB9rOefa/X7/OcuyUuOEAqe0eO97hr1rjVA/kWXZRaa0BOwbvHddcTMFE/hGjUysozoJNSKhozrn8/lKVZ9EpPTaN5WFTqgRCRnVOZlMbofD4UOe53c+kYtU+8SBUCsQcPl1r6+vzdD9fUZ1FkWR+kYuUv0Th2hDDfyt5h1MyOV3t9sl+/3++2az+SUivnf9QQLm0YpU+MQhylBDf6uFBBNy+R2Px+3RaHT98vLSKHXCNVHljVyUoYbEInKeYHwvv777XVrV71yIMtR3vpcxy8FcStXvXIg6VHy9qt65cFKoi8UiKbvrarVKRETW63UynU6vv3K91b2tnnfo+lP6+gMnXrDyeeKrmgAAAABJRU5ErkJggg=="">",0(0.0%)
3,channel_id[int64],Mean (sd) : 7.9 (8.4)min < med < max:1.0 < 5.0 < 49.0IQR (CV) : 0.0 (0.9),39 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACaUlEQVR4nO3XMU/bQBiH8fdtILVpekeI6pGVoVI+gPeufFjWbh0ydGRnR4gAtgpOcY07VOpSCfCFKPlXz2/Oe74oz50V7/vegF33btsbAF5j76UPuHtuZuPE9R/7vm8SZ4G/ng3V3fPZbHYaQpimLF7X9a27nxEr1vXSjToOIUzLsmxijKshC1dVlS0Wi+lyuRybGaFiLS+++s3MYoyroigeEtbPE2aAf/BnChIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRL2tr2B57h7bmbjxPHHvu+bt9wPtmdnQ3X3fDabnYYQpinzdV3fuvsZsf4fdjZUMxuHEKZlWTYxxtWQwaqqssViMV0ul2MzSwqV23y3bDTUruv2zSy4e8p4eHp62o8x3hZF8ZAwn6c81IzbPNUmD/fGQr2/v99v23Z+fHw8Go1Gg25EM7O2bbOu6z6vVqtvZjY41HUPyWQyKcqyrLdxm69jW2+CTR/uV4VaVVU29ME3NzeTLMsOTk5O/PDw8OfQ+evr6+zi4uKgqqoPV1dXj0NmLy8vP77FIWnb9ruZDZ7vum7P0g/JOrKjo6MvMcZJyvDd3d0Pd/9qCd/Z/hzuT/P5/Fee54N+76Zp3p+fnz97uH8D/57Am47yZQoAAAAASUVORK5CYII="">",0(0.0%)
4,payment_order_id[int64],Mean (sd) : 82311715.8 (7010887.3)min < med < max:68405119.0 < 83228842.0 < 93139817.0IQR (CV

In [18]:
#Visualizando um pouco da base pós cruzamento
database[20:25].T

,20,21,22,23,24
order_id,68414309,68414512,68414563,68414907,68415103
store_id,236,631,955,294,631
channel_id,5,5,5,5,5
payment_order_id,68414309,68414512,68414563,68414907,68415103
delivery_order_id,68414309,68414512,68414563,68414907,68415103
order_status,FINISHED,FINISHED,FINISHED,CANCELED,FINISHED
order_amount,94.9,45.9,26.9,32.4,103.6
order_delivery_fee,11.9,11.9,0.0,9.9,11.9
order_delivery_cost,6.0,11.63,6.0,NaN,6.33
order_created_hour,14,14,14,14,14


In [36]:
round(database.loc[:, database.isnull().mean()>0].isnull().mean().sort_values(ascending=False) * 100, 1)

order_moment_delivered               94.5
store_plan_price                     23.0
order_metric_walking_time            19.8
order_metric_paused_time             19.0
order_moment_in_expedition           17.9
order_metric_collected_time          13.8
order_moment_collected               11.4
order_metric_expediton_speed_time     9.2
order_metric_transit_time             6.8
order_metric_production_time          6.7
order_moment_ready                    6.7
order_moment_delivering               6.6
delivery_distance_meters              6.4
delivery_id                           6.4
driver_modal                          6.4
delivery_status                       6.4
driver_id                             6.4
driver_type                           6.4
order_moment_finished                 4.0
order_metric_cycle_time               4.0
order_moment_accepted                 2.4
order_delivery_cost                   2.2
store_longitude                       0.4
store_latitude                    